In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
import os


In [3]:
conn = Neo4jConnection(uri="bolt://"+ os.environ['NEO_IP']+":7687", user=os.environ['NEO_NAME'], pwd=os.environ['NEO_PASS'])


In [4]:
result = conn.query("MATCH p=()-[r:HAS_COMMON]->() where r.count > 2000  RETURN p")


In [5]:
type(result)

list